# Image to ASCII Generator
---
- Author: Diego Inácio
- GitHub: [github.com/diegoinacio](https://github.com/diegoinacio)
- Notebook: [ascii-generator.ipynb](https://github.com/diegoinacio/creative-coding-notebooks/blob/master/Generative/ascii-generator.ipynb)
---
Implementation of a simple image to ASCII generator.

In [ ]:
from PIL import (
    Image, 
    ImageOps, 
    ImageFont, 
    ImageDraw
)

import numpy as np

from IPython.display import HTML, display

## Symbols
---

In [ ]:
# Define symbol set
symbols = list("!@#%¨&*()-_=+{}[]<>^~,.:;?|")
symbols += [chr(e) for e in range(ord("a"), ord("z") + 1)]
symbols += [chr(e) for e in range(ord("A"), ord("Z") + 1)]
symbols += list("0123456789")
N = len(symbols)

In [ ]:
# Download font
import requests
URL = "https://github.com/googlefonts/roboto/blob/master/src/hinted/Roboto-Regular.ttf?raw=true"
response = requests.get(URL)
open("Roboto-Regular.ttf", "wb").write(response.content)

In [ ]:
# Raster symbol set parameters
S, T = 32, 32 # raster sizes

# Create raster symbol set
SYMBOLS = np.zeros((N, T, S))
for i, e in enumerate(symbols):
    image = Image.fromarray(np.zeros((S, S)))
    draw = ImageDraw.Draw(image)
    font = ImageFont.truetype("Roboto-Regular.ttf", S)
    draw.text((S/2, S/2), e, anchor="mm", font=font)
    SYMBOLS[i] = np.asarray(image)

## Input B&W
---

In [ ]:
# Read input B&W image
IMAGE_BW = Image.open("../_data/cameraman.png")

# Histogram equalization
IMAGE_BW = ImageOps.equalize(IMAGE_BW)

# Show image
IMAGE_BW

In [ ]:
# Ascii symbols parameters
m, n = 160, 80 # number of ascii symbols per axis

# Resize input image based on inputs
IMAGE_ = IMAGE_BW.resize((m*S, n*T))
IMAGE_ = np.asarray(IMAGE_)/255

# Build ascii conversions
OUTPUT = [["" for _ in range(m)] for _ in range(n)]
for j, ROW in enumerate(OUTPUT):
    for i, e in enumerate(ROW):
        image = IMAGE_[j*S:(j+1)*S, i*S:(i+1)*S]
        diff = np.power(SYMBOLS - image, 2).reshape((N, -1))
        OUTPUT[j][i] = symbols[np.argmin(diff.sum(axis=1))]

In [ ]:
STYLE = """
<style>
  .ascii-text-block-bw {
    display: block;
    word-wrap:  normal;
    background-color: black;
    font-family: monospace;
    font-size: 8px;
    letter-spacing: 1px;
    line-height: 30%;
    padding: 1em;
    top: 0;
    left: 0;
    mix-blend-mode: screen;
    color: white;
  }

  .ascii-text-container-bw {
    background-color: black;
    display: inline-block;
  }
</style>
"""

# Build HTML
DIV = f'<div class="ascii-text-container-bw">'

DIV += f'<div class="ascii-text-block-bw">'
OUTPUT_ROWS = ["".join(e) for e in OUTPUT]
DIV += "".join([f'<p>{row}</p>' for row in OUTPUT_ROWS])
DIV += "</div>"

DIV += "</div>"

# Display ascii art
display(HTML(STYLE + DIV))

In [ ]:
# Print HTML div
# print(STYLE + DIV)

## Input RGB
---

In [ ]:
# Read input RGB image
IMAGE_RGB = Image.open("../_data/woman03.png")

# Histogram equalization
IMAGE_RGB = ImageOps.equalize(IMAGE_RGB)

# Convert to HSV
IMAGE_HSV = IMAGE_RGB.convert("HSV")

# Show image
IMAGE_RGB

In [ ]:
# Ascii symbols parameters
m, n = 80, 40 # number of ascii symbols per axis

# Resize input image based on inputs
IMAGE_ = IMAGE_HSV.resize((m*S, n*T))
IMAGE_ = np.asarray(IMAGE_)/255

# Build ascii conversions
OUTPUT = [[{"chr": "", "hue": ""} for _ in range(m)] for _ in range(n)]
for j, ROW in enumerate(OUTPUT):
    for i, e in enumerate(ROW):
        image = IMAGE_[j*S:(j+1)*S, i*S:(i+1)*S, 2]
        diff = np.power(SYMBOLS - image, 2).reshape((N, -1))
        OUTPUT[j][i]["chr"] = symbols[np.argmin(diff.sum(axis=1))]
        block_hue = int(np.median(IMAGE_[j*S:(j+1)*S, i*S:(i+1)*S, 0])*360)
        block_sat = int(np.median(IMAGE_[j*S:(j+1)*S, i*S:(i+1)*S, 1])*100)
        block_val = int(np.median(IMAGE_[j*S:(j+1)*S, i*S:(i+1)*S, 2])*100)
        OUTPUT[j][i]["color"] = f'hsl({block_hue}, {block_sat}%, {block_val}%)'

In [ ]:
STYLE = """
<style>
  .ascii-text-block-rgb {
    display: block;
    word-wrap:  normal;
    font-family: monospace;
    font-size: 16px;
    letter-spacing: 1px;
    line-height: 30%;
    padding: 1em;
    top: 0;
    left: 0;
    mix-blend-mode: screen;
    color: white;
  }

  .ascii-text-container-rgb {
    background-color: black;
    display: inline-block;
  }
</style>
"""

# Build HTML
DIV = f'<div class="ascii-text-container-rgb">'

DIV += f'  <div class="ascii-text-block-rgb">'
OUTPUT_ROWS = ["".join(
    [f'<span style="color: {e["color"]};">{e["chr"]}</span>' for e in row]
) for row in OUTPUT]
DIV += "".join([f'<p>{row}</p>' for row in OUTPUT_ROWS])
DIV += "</div>"

DIV += "</div>"

# Display ascii art
display(HTML(STYLE + DIV))

In [ ]:
# Print HTML div
# print(STYLE + DIV)